In [4]:
from transformers import pipeline

# load zero-shot classifier
classifier = pipeline("zero-shot-classification",model="sileod/deberta-v3-base-tasksource-nli")

In [1]:
import pandas as pd
import numpy as np

# load two datasets
df_preprocessed = pd.read_csv("./osr_tweets_without_T_U_U_v2.csv", engine='python')
df_without_preprocessed = pd.read_csv("./osr_tweets_origin_v2.csv", engine='python')

df_preprocessed.text = df_preprocessed.text.astype(str)
df_preprocessed = df_preprocessed[df_preprocessed['text'].map(len) >= 10]

In [2]:
# collect target topics
candidate_labels = []
for label in df_without_preprocessed.topic:
    if label not in candidate_labels:
        candidate_labels.append(label)

In [8]:
from tqdm import tqdm
pre_output_list = []
pre_groundtruth_list = []
output_list = []
groundtruth_list = []
for i in tqdm(range(1, len(df_preprocessed), 5), total=len(df_preprocessed)//5):
    text = df_preprocessed.text.iloc[i]
    predict_label = classifier(text, candidate_labels)
    pre_output_list.append(predict_label['labels'][0])
    pre_groundtruth_list.append(df_preprocessed.topic.iloc[i])
    
for i in tqdm(range(1, len(df_without_preprocessed), 5), total=len(df_without_preprocessed)//5):
    text = df_without_preprocessed.text.iloc[i]
    predict_label = classifier(text, candidate_labels)
    output_list.append(predict_label['labels'][0])
    groundtruth_list.append(df_without_preprocessed.topic.iloc[i])

print("Accuracy of Zero-shot without preprocessing:", np.mean(np.array(output_list) == groundtruth_list))
print("Accuracy of Zero-shot with preprocessing:", np.mean(np.array(pre_output_list) == pre_groundtruth_list))

2037it [53:41,  1.58s/it]                                                       
2058it [1:05:42,  1.92s/it]                                                     

Accuracy of Zero-shot without preprocessing: 0.293488824101069
Accuracy of Zero-shot with preprocessing: 0.18311242022582228
